In [31]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectKBest,f_classif,mutual_info_classif,chi2
import sys

In [32]:
sys.path.append('../')

import helper

In [33]:
class AutoPosModel:
    def __init__(self,Aligned_Enzyme_Datafile,scoring_func,savedfilename,n_pos):
        self.X,self.y,self.enz_names = helper.parseEnzymeFile(Aligned_Enzyme_Datafile)
        self.scfunc = scoring_func
        self.filename =  savedfilename
        self.n_pos = n_pos
        pass
    
class OhePosModel(AutoPosModel):
    
    def __init__(self,Aligned_Enzyme_Datafile,scoring_func,savedfilename,n_pos):
        super().__init__(Aligned_Enzyme_Datafile,scoring_func,savedfilename,n_pos)
        
        self.ohe_enc = self._getOHEncoder(self.X)
        self.X_enc = self.ohe_enc.transform(self.X).toarray()
        self.skbest = self._getBestMapperObj(self.X_enc,self.y,self.scfunc)
        self.lencat = self._getSeqPos2nAAmap(self.ohe_enc)
        self.numcatmap = self._getOHEpos2seqpos(self.lencat)
        self.BestPositions = self._getBestPositions(self.skbest,self.numcatmap,self.n_pos)
        self._savebestpos(self.BestPositions,self.filename)
        
        
    
    def _getOHEncoder(self,Xarr):
        '''A one hot encoded representation of the aligned enzyme sequences'''
        ohe = OneHotEncoder()
        ohe.fit(Xarr)
        return ohe
    
    def _getSeqPos2nAAmap(self,ohe):
        '''Number of AAs identified per postion; Functions to be used when mapping back the OHE encoded position 
        to the actual position in the aligned sequence of the enzyme'''
        lencat = [(ci,lc) for ci,lc in zip(range(len(ohe.categories_)),list(map(len,ohe.categories_)))]
        return lencat
    
    def _getOHEpos2seqpos(self,lencat):
        '''mapping from the OHE encoded sequence to the aligned sequence position'''
        
        catnummap = {}
        currval = 0
        nextval = 0

        for i,j in lencat:
            nextval += j
            catnummap[i] = list(range(currval,nextval))
            currval=nextval
            
        numcatmap = {num:cat for cat,nums in catnummap.items() for num in nums}
        return numcatmap
    
    def _getBestMapperObj(self,X_ohe_enc,y,scoring_func):
        skbest = SelectKBest(score_func=scoring_func)
        skbest.fit(X_ohe_enc,y)
        return skbest
    
    def _getBestPositions(self,skbest,numcatmap,n_pos=50):
        
        bestpos = set()
        i=0
        lenbestpos = 0
        while lenbestpos<n_pos:
            bestpos.add(numcatmap[np.argsort(skbest.scores_)[i]])
            i+=1
            lenbestpos = len(bestpos)
            
        return sorted(bestpos)
    
    def _savebestpos(self,bestpos,filename):
        with open(filename,'w') as f:
            for pos in bestpos:
                f.write(str(pos))
                f.write('\n')
            
        return  
    
    
    

    
class AEPosModel(AutoPosModel):
    def __init__(self,Aligned_Enzyme_Datafile,scoring_func,savedfilename,n_pos,runAE=False):
        super().__init__(Aligned_Enzyme_Datafile,scoring_func,savedfilename,n_pos)
            
        self.AAs = ['-','A','C','D','E','F','G','H','I','K','L',
                    'M','N','P','Q','R','S','T','V','W','X','Y']
        self.GAAmap = {'G':'g1','A':'g1','V':'g1','L':'g1','M':'g1','I':'g1','F':'g2','Y':'g2','W':'g2',
                       'K':'g3','R':'g3','H':'g3', 'D':'g4', 'E':'g4','S':'g5','T':'g5','C':'g5','P':'g5',
                       'N':'g5','Q':'g5','X':'g6','-':'g6'}
        self.ohedict = self._get_ohe_map()
        
        if runAE:
            '''run NN based AE'''
            self.Xtrain = self._create_train_set()
            self._getAE(self.Xtrain)
        
        else:
            '''Import saved model'''
            savedmodel = 'AEModel.sav'
            self.reg = pickle.load(open(savedmodel,'rb'))

        self.Xtest = self._create_test_set()
        self.encode_dict = self._getEncodedMap(self.Xtest)
        self.X_enc = np.array(list(map(self.autoencode,self.X)))
        self.skbest = self._getBestMapperObj(self.X_enc,self.y,self.scfunc)
        self.BestPositions = self._getBestPositions(self.skbest,self.n_pos)
        self._savebestpos(self.BestPositions,self.filename)
        
        
    def _get_ohe_map(self):
        OHEdict = {aa:num for aa, num in zip(self.AAs,range(len(self.AAs)))}
        GAAlist = sorted(set(self.GAAmap.values()))
        GAAnum = range(len(OHEdict),len(OHEdict)+len(GAAlist))
        for gaa,gnum in zip(GAAlist,GAAnum):
            OHEdict[gaa] = gnum
        return OHEdict
    
    def _get_OHE_val(self,aa):
        ohe = [0 for i in range(len(self.ohedict))]
        ohe[self.ohedict[aa]] = 1
        ohe[self.ohedict[self.GAAmap[aa]]] = 1
        return ohe
    
    def _create_train_set(self):
        AAs = [aa for i in range(2000) for aa in self.AAs]
        Xtrain = list(map(self._get_OHE_val,AAs))
        return np.array(Xtrain)

    def _create_test_set(self):
        Xtest = list(map(self._get_OHE_val,self.AAs))
        return np.array(Xtest)
    
    def _predict_AAs(self,predArr):
        AApredictionidx = np.argmax(predArr[:22])
        return self.AAs[AApredictionidx]
    
    def _getAE(self,Xtrain):
        hls = (25,21,15,11,7,3,1,3,7,11,15,23,26)
        self.reg = MLPRegressor(hidden_layer_sizes=hls
                   ,batch_size=10000,
                   activation = 'tanh',solver='adam',learning_rate='adaptive',
                   learning_rate_init=0.001,max_iter=5000,tol=1e-7,verbose=True,
                  n_iter_no_change=500,alpha=0.01)
        self.reg.fit(Xtrain,Xtrain)
        return 

    def _getEncodedMap(self,Xtest):
        AApred = self.reg.predict(Xtest)
        self.predictions = list(map(self._predict_AAs,AApred))
        AA2Contdict = dict(zip(self.AAs,np.ravel(self.encoder(Xtest,6))))
        return AA2Contdict

    def tanh(self,x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def encode(self,x,layer):
        linear_layer = x*self.reg.coefs_[layer] + self.reg.intercepts_[layer]
        activation = self.tanh(linear_layer)
        return activation

    def encoder(self,data,nlayers):
        data = np.asmatrix(data)
        encoder = data
        for i in range(nlayers):
            encoder = self.encode(encoder,i)
        latent = self.encode(encoder,nlayers)
        return np.asarray(latent)
    
    def autoencode(self,seq):
        x_autoencoded = []
        for aa in seq:
            x_autoencoded.append(self.encode_dict[aa])
        return np.array(x_autoencoded)
    
    
    def _getBestMapperObj(self,X_enc,y,scoring_func):
        skbest = SelectKBest(score_func=scoring_func)
        skbest.fit(X_enc,y)
        return skbest 
        
    def _getBestPositions(self,skbest,n_pos=50):
        bestpos = set()
        i=0
        lenbestpos = 0
        while lenbestpos<n_pos:
            bestpos.add(np.argsort(skbest.scores_)[i])
            i+=1
            lenbestpos = len(bestpos)
        return sorted(bestpos)
    
    
    def _savebestpos(self,bestpos,filename):
        with open(filename,'w') as f:
            for pos in bestpos:
                f.write(str(pos))
                f.write('\n')
            
        return  
      

In [34]:
Aligned_Data_File = '../Data/Enzyme_aligned.txt'
sc_func4 = f_classif
sc_func5 = mutual_info_classif
save_file4 = 'BestPos4.txt'
save_file5 = 'BestPos5.txt'
n_positions=50

aepos_model = AEPosModel(Aligned_Data_File,sc_func4,save_file4,n_positions,runAE=True)


Iteration 1, loss = 0.08200006
Iteration 2, loss = 0.06032507
Iteration 3, loss = 0.04915830
Iteration 4, loss = 0.04321257
Iteration 5, loss = 0.03927713
Iteration 6, loss = 0.03618753
Iteration 7, loss = 0.03382435
Iteration 8, loss = 0.03209266
Iteration 9, loss = 0.03090955
Iteration 10, loss = 0.03008861
Iteration 11, loss = 0.02951316
Iteration 12, loss = 0.02910556
Iteration 13, loss = 0.02879986
Iteration 14, loss = 0.02854797
Iteration 15, loss = 0.02832149
Iteration 16, loss = 0.02811531
Iteration 17, loss = 0.02793257
Iteration 18, loss = 0.02776844
Iteration 19, loss = 0.02762440
Iteration 20, loss = 0.02749808
Iteration 21, loss = 0.02738438
Iteration 22, loss = 0.02727704
Iteration 23, loss = 0.02716958
Iteration 24, loss = 0.02705691
Iteration 25, loss = 0.02693329
Iteration 26, loss = 0.02680147
Iteration 27, loss = 0.02666967
Iteration 28, loss = 0.02654076
Iteration 29, loss = 0.02640988
Iteration 30, loss = 0.02627664
Iteration 31, loss = 0.02613861
Iteration 32, los

Iteration 253, loss = 0.01163967
Iteration 254, loss = 0.01164235
Iteration 255, loss = 0.01160413
Iteration 256, loss = 0.01159979
Iteration 257, loss = 0.01157239
Iteration 258, loss = 0.01154996
Iteration 259, loss = 0.01153188
Iteration 260, loss = 0.01151567
Iteration 261, loss = 0.01149862
Iteration 262, loss = 0.01148097
Iteration 263, loss = 0.01146605
Iteration 264, loss = 0.01145167
Iteration 265, loss = 0.01143724
Iteration 266, loss = 0.01142432
Iteration 267, loss = 0.01141014
Iteration 268, loss = 0.01139789
Iteration 269, loss = 0.01139618
Iteration 270, loss = 0.01140231
Iteration 271, loss = 0.01136676
Iteration 272, loss = 0.01134974
Iteration 273, loss = 0.01133438
Iteration 274, loss = 0.01131998
Iteration 275, loss = 0.01131177
Iteration 276, loss = 0.01131942
Iteration 277, loss = 0.01129463
Iteration 278, loss = 0.01127857
Iteration 279, loss = 0.01125953
Iteration 280, loss = 0.01124326
Iteration 281, loss = 0.01123803
Iteration 282, loss = 0.01121067
Iteration 

Iteration 502, loss = 0.00553693
Iteration 503, loss = 0.00550372
Iteration 504, loss = 0.00547530
Iteration 505, loss = 0.00544486
Iteration 506, loss = 0.00541919
Iteration 507, loss = 0.00539420
Iteration 508, loss = 0.00537578
Iteration 509, loss = 0.00534608
Iteration 510, loss = 0.00532064
Iteration 511, loss = 0.00532168
Iteration 512, loss = 0.00533921
Iteration 513, loss = 0.00548718
Iteration 514, loss = 0.00541835
Iteration 515, loss = 0.00531879
Iteration 516, loss = 0.00551096
Iteration 517, loss = 0.00537919
Iteration 518, loss = 0.00524573
Iteration 519, loss = 0.00521187
Iteration 520, loss = 0.00523091
Iteration 521, loss = 0.00527905
Iteration 522, loss = 0.00523552
Iteration 523, loss = 0.00512592
Iteration 524, loss = 0.00510433
Iteration 525, loss = 0.00504434
Iteration 526, loss = 0.00500466
Iteration 527, loss = 0.00498920
Iteration 528, loss = 0.00496891
Iteration 529, loss = 0.00494191
Iteration 530, loss = 0.00492621
Iteration 531, loss = 0.00491242
Iteration 

Iteration 751, loss = 0.00327571
Iteration 752, loss = 0.00326324
Iteration 753, loss = 0.00325105
Iteration 754, loss = 0.00324285
Iteration 755, loss = 0.00327381
Iteration 756, loss = 0.00325728
Iteration 757, loss = 0.00320838
Iteration 758, loss = 0.00323150
Iteration 759, loss = 0.00321338
Iteration 760, loss = 0.00319221
Iteration 761, loss = 0.00329268
Iteration 762, loss = 0.00327122
Iteration 763, loss = 0.00323759
Iteration 764, loss = 0.00318027
Iteration 765, loss = 0.00314814
Iteration 766, loss = 0.00311841
Iteration 767, loss = 0.00310102
Iteration 768, loss = 0.00309444
Iteration 769, loss = 0.00305699
Iteration 770, loss = 0.00305130
Iteration 771, loss = 0.00303226
Iteration 772, loss = 0.00301783
Iteration 773, loss = 0.00301974
Iteration 774, loss = 0.00308866
Iteration 775, loss = 0.00299779
Iteration 776, loss = 0.00307816
Iteration 777, loss = 0.00297384
Iteration 778, loss = 0.00297158
Iteration 779, loss = 0.00297637
Iteration 780, loss = 0.00294115
Iteration 

Iteration 1000, loss = 0.00185514
Iteration 1001, loss = 0.00185555
Iteration 1002, loss = 0.00185452
Iteration 1003, loss = 0.00185392
Iteration 1004, loss = 0.00185396
Iteration 1005, loss = 0.00185342
Iteration 1006, loss = 0.00185288
Iteration 1007, loss = 0.00185340
Iteration 1008, loss = 0.00185308
Iteration 1009, loss = 0.00185254
Iteration 1010, loss = 0.00185283
Iteration 1011, loss = 0.00185344
Iteration 1012, loss = 0.00185537
Iteration 1013, loss = 0.00186547
Iteration 1014, loss = 0.00188172
Iteration 1015, loss = 0.00185536
Iteration 1016, loss = 0.00188000
Iteration 1017, loss = 0.00186722
Iteration 1018, loss = 0.00186167
Iteration 1019, loss = 0.00185297
Iteration 1020, loss = 0.00185638
Iteration 1021, loss = 0.00184856
Iteration 1022, loss = 0.00185057
Iteration 1023, loss = 0.00184915
Iteration 1024, loss = 0.00184571
Iteration 1025, loss = 0.00185114
Iteration 1026, loss = 0.00184967
Iteration 1027, loss = 0.00184516
Iteration 1028, loss = 0.00184555
Iteration 1029

...

Iteration 1723, loss = 0.00024921
Iteration 1724, loss = 0.00024912
Iteration 1725, loss = 0.00024906
Iteration 1726, loss = 0.00024900
Iteration 1727, loss = 0.00024897
Iteration 1728, loss = 0.00024895
Iteration 1729, loss = 0.00024893
Iteration 1730, loss = 0.00024891
Iteration 1731, loss = 0.00024889
Iteration 1732, loss = 0.00024888
Iteration 1733, loss = 0.00024887
Iteration 1734, loss = 0.00024886
Iteration 1735, loss = 0.00024885
Iteration 1736, loss = 0.00024884
Iteration 1737, loss = 0.00024883
Iteration 1738, loss = 0.00024882
Iteration 1739, loss = 0.00024881
Iteration 1740, loss = 0.00024880
Iteration 1741, loss = 0.00024880
Iteration 1742, loss = 0.00024879
Iteration 1743, loss = 0.00024878
Iteration 1744, loss = 0.00024877
Iteration 1745, loss = 0.00024877
Iteration 1746, loss = 0.00024876
Iteration 1747, loss = 0.00024875
Iteration 1748, loss = 0.00024875
Iteration 1749, loss = 0.00024874
Iteration 1750, loss = 0.00024873
Iteration 1751, loss = 0.00024873
Iteration 1752

Iteration 1964, loss = 0.00024851
Iteration 1965, loss = 0.00024856
Iteration 1966, loss = 0.00024820
Iteration 1967, loss = 0.00024803
Iteration 1968, loss = 0.00024847
Iteration 1969, loss = 0.00024777
Iteration 1970, loss = 0.00024789
Iteration 1971, loss = 0.00024805
Iteration 1972, loss = 0.00024800
Iteration 1973, loss = 0.00024794
Iteration 1974, loss = 0.00024787
Iteration 1975, loss = 0.00024772
Iteration 1976, loss = 0.00024770
Iteration 1977, loss = 0.00024769
Iteration 1978, loss = 0.00024765
Iteration 1979, loss = 0.00024770
Iteration 1980, loss = 0.00024805
Iteration 1981, loss = 0.00025089
Iteration 1982, loss = 0.00025056
Iteration 1983, loss = 0.00025022
Iteration 1984, loss = 0.00025334
Iteration 1985, loss = 0.00024963
Iteration 1986, loss = 0.00025308
Iteration 1987, loss = 0.00025176
Iteration 1988, loss = 0.00024983
Iteration 1989, loss = 0.00025015
Iteration 1990, loss = 0.00024810
Iteration 1991, loss = 0.00024937
Iteration 1992, loss = 0.00024788
Iteration 1993

Iteration 2205, loss = 0.00024658
Iteration 2206, loss = 0.00024662
Iteration 2207, loss = 0.00024670
Iteration 2208, loss = 0.00024684
Iteration 2209, loss = 0.00024733
Iteration 2210, loss = 0.00025068
Iteration 2211, loss = 0.00026816
Iteration 2212, loss = 0.00031723
Iteration 2213, loss = 0.00027443
Iteration 2214, loss = 0.00028119
Iteration 2215, loss = 0.00027175
Iteration 2216, loss = 0.00026188
Iteration 2217, loss = 0.00025518
Iteration 2218, loss = 0.00025074
Iteration 2219, loss = 0.00024873
Iteration 2220, loss = 0.00024790
Iteration 2221, loss = 0.00024749
Iteration 2222, loss = 0.00024733
Iteration 2223, loss = 0.00024724
Iteration 2224, loss = 0.00024691
Iteration 2225, loss = 0.00024669
Iteration 2226, loss = 0.00024657
Iteration 2227, loss = 0.00024656
Iteration 2228, loss = 0.00024654
Iteration 2229, loss = 0.00024649
Iteration 2230, loss = 0.00024648
Iteration 2231, loss = 0.00024647
Iteration 2232, loss = 0.00024646
Iteration 2233, loss = 0.00024645
Iteration 2234

Iteration 2446, loss = 0.00024702
Iteration 2447, loss = 0.00024701
Iteration 2448, loss = 0.00024701
Iteration 2449, loss = 0.00024701
Iteration 2450, loss = 0.00024701
Iteration 2451, loss = 0.00024700
Iteration 2452, loss = 0.00024700
Iteration 2453, loss = 0.00024700
Iteration 2454, loss = 0.00024700
Iteration 2455, loss = 0.00024699
Iteration 2456, loss = 0.00024699
Iteration 2457, loss = 0.00024699
Iteration 2458, loss = 0.00024698
Iteration 2459, loss = 0.00024698
Iteration 2460, loss = 0.00024698
Iteration 2461, loss = 0.00024698
Iteration 2462, loss = 0.00024697
Iteration 2463, loss = 0.00024697
Iteration 2464, loss = 0.00024697
Iteration 2465, loss = 0.00024697
Iteration 2466, loss = 0.00024696
Iteration 2467, loss = 0.00024696
Iteration 2468, loss = 0.00024696
Iteration 2469, loss = 0.00024695
Iteration 2470, loss = 0.00024695
Iteration 2471, loss = 0.00024695
Iteration 2472, loss = 0.00024695
Iteration 2473, loss = 0.00024694
Iteration 2474, loss = 0.00024694
Iteration 2475

/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [197 204 211 226 240 269 346 427 437] are constant.
  UserWarning)
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [35]:
aepos_model.predictions

['-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'X',
 'Y']

In [16]:
if __name__=='__main__':
    np.random.seed(77)
    Aligned_Data_File = '../Data/Enzyme_aligned.txt'
    sc_func = f_classif
    sc_func2 = chi2
    sc_func3 = mutual_info_classif
    save_file = 'BestPos1.txt'
    save_file2 = 'BestPos2.txt'
    save_file3 = 'BestPos3.txt'
    save_file4 = 'BestPos4.txt'
    save_file5 = 'BestPos5.txt'

    n_positions=50

    ohepos = OhePosModel(Aligned_Data_File,sc_func,save_file,n_positions)
    ohepos_chi = OhePosModel(Aligned_Data_File,sc_func2,save_file2,n_positions)
    ohepos_mi = OhePosModel(Aligned_Data_File,sc_func3,save_file3,100)
    aepos_model = AEPosModel(Aligned_Data_File,sc_func,save_file4,n_positions)
    aepos_model_mi = AEPosModel(Aligned_Data_File,sc_func3,save_file5,100)

/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [ 573  580  585  621  631  632  644  693  714  735  739  740  781  782
  788  843  923  948  966 1007 1197 1210 1213 1217 1220 1221 1225 1233
 1240 1257 1332 1333 1334 1335 1336 1365 1370 1548 1588 1618 1626] are constant.
  UserWarning)
/storage/home/dzb5732/.conda/envs/thio-env/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
